### Loading the Data

In [62]:
# Import pandas and load the raw cafe sales data
import pandas as pd

file_path = '../data/raw/dirty_cafe_sales.csv'
df = pd.read_csv(file_path)


### Initial Data Audit

In [63]:
# View first 10 rows   columns, and shape
df.head(10)

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11
5,TXN_2602893,Smoothie,5,4.0,20.0,Credit Card,NaN,2023-03-31
6,TXN_4433211,UNKNOWN,3,3.0,9.0,ERROR,Takeaway,2023-10-06
7,TXN_6699534,Sandwich,4,4.0,16.0,Cash,UNKNOWN,2023-10-28
8,TXN_4717867,NaN,5,3.0,15.0,NaN,Takeaway,2023-07-28
9,TXN_2064365,Sandwich,5,4.0,20.0,NaN,In-store,2023-12-31


In [64]:
# check info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


In [65]:
# summary stats
df.describe()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
count,10000,9667,9862,9821,9827,7421,6735,9841
unique,10000,10,7,8,19,5,4,367
top,TXN_1961373,Juice,5,3.0,6.0,Digital Wallet,Takeaway,UNKNOWN
freq,1,1171,2013,2429,979,2291,3022,159


In [66]:
# View columns
df.columns

Index(['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
       'Payment Method', 'Location', 'Transaction Date'],
      dtype='object')

In [67]:
# View Shape 
df.shape

(10000, 8)

### Cleaning

In [68]:
# Rename columns: strip whitespace and rename 'Total Spent' to 'Total Purchase'
df.columns = df.columns.str.strip().str.replace("Total Spent","Total Purchase")


In [69]:
# Drop the Irrelivent columns
df.drop(columns='Transaction ID', inplace=True)

In [70]:
# Converting Transaction Date to datetime from string
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'],format='%Y-%m-%d',errors='coerce')


In [71]:
# Converting the appropiate columns from string to numeric
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Quantity'] = pd.to_numeric(df['Quantity'],errors='coerce')
df['Total Purchase'] = pd.to_numeric(df['Total Purchase'],errors='coerce')

# Checking for the data types
df.dtypes

Item                        object
Quantity                   float64
Price Per Unit             float64
Total Purchase             float64
Payment Method              object
Location                    object
Transaction Date    datetime64[ns]
dtype: object

In [72]:
# Checking missing values in each column
df.isna().sum()

Item                 333
Quantity             479
Price Per Unit       533
Total Purchase       502
Payment Method      2579
Location            3265
Transaction Date     460
dtype: int64

In [ ]:
# Converting the ERROR and UNKNOWN to NaN values
import numpy as np

df = df.replace(
    to_replace=r'(?i)^\s*(ERROR|UNKNOWN)\s*$',
    value=np.nan,
    regex=True
)

In [ ]:
# Rechecking the NaN values 
df.isna().sum()

Item                 969
Quantity             479
Price Per Unit       533
Total Purchase       502
Payment Method      3178
Location            3961
Transaction Date     460
dtype: int64

#### Filling up the common NaN values

In [ ]:
# Filling the NaN value of Total Purchase
rows = df['Total Purchase'].isna() & df['Quantity'].notna() & df['Price Per Unit'].notna()
df.loc[rows,'Total Purchase'] = df.loc[rows,'Quantity'] * df.loc[rows,'Price Per Unit']

In [ ]:
# Filling the NaN values of Price Per Unit
rows = df['Price Per Unit'].isna() & df['Total Purchase'].notna() & df['Quantity'].notna()
df.loc[rows,'Price Per Unit'] = df.loc[rows,'Total Purchase'] / df.loc[rows,'Quantity']

In [ ]:
# Filling the Nan Values of Quantity
rows = df['Quantity'].isna() & df['Price Per Unit'].notna() & df['Total Purchase'].notna()
df.loc[rows,'Quantity'] = df.loc[rows,'Total Purchase'] / df.loc[rows,'Price Per Unit']

In [ ]:
# Checking for remaining NaN values after filling
print(f"NaN in Total Purchase: {df['Total Purchase'].isna().sum()}")
print(f"NaN in Quantity: {df['Quantity'].isna().sum()}")
print(f"NaN in Price Per Unit: {df['Price Per Unit'].isna().sum()}")

NaN in Total Purchase: 40
NaN in Quantity: 38
NaN in Price Per Unit: 38


#### Filling up the Item NaN values

In [ ]:
# checking for Items along with there individual prices
df.groupby(['Item','Price Per Unit']).size().reset_index(name='Count').sort_values('Count',ascending=False)


,Item,Price Per Unit,Count
3,Juice,3.0,1170
1,Coffee,2.0,1163
4,Salad,5.0,1146
0,Cake,3.0,1135
5,Sandwich,4.0,1126
6,Smoothie,4.0,1091
2,Cookie,1.0,1086
7,Tea,1.5,1082


In [ ]:
# Checking for total no. of NaN in Item
print(f"The Total NaN in {df['Item'].isna().sum()}")

The Total NaN in 969


In [ ]:
# Filling up the missing item's as per there prices
df.loc[df['Item'].isna() & (df['Price Per Unit'] == 2.0),'Item'] = 'Coffee'
df.loc[df['Item'].isna() & (df['Price Per Unit'] == 5.0),'Item'] = 'Salad'
df.loc[df['Item'].isna() & (df['Price Per Unit'] == 1.0),'Item'] = 'Cookie'
df.loc[df['Item'].isna() & (df['Price Per Unit'] == 1.5), 'Item'] = 'Tea'

In [ ]:
# Rechecking the missing values in Item again 
df.groupby(['Item','Price Per Unit']).size().reset_index(name="Count").sort_values('Count',ascending=False)

,Item,Price Per Unit,Count
1,Coffee,2.0,1289
4,Salad,5.0,1270
2,Cookie,1.0,1207
7,Tea,1.5,1200
3,Juice,3.0,1170
0,Cake,3.0,1135
5,Sandwich,4.0,1126
6,Smoothie,4.0,1091


In [84]:
print(f"The Total NaN values in Item: {df['Item'].isna().sum()}")

The Total NaN values in Item: 480


### Filling the NaN values for price per unit - 4.0

In [ ]:
# Filtering Out the Item which have price per unit - 4.0
known = (
    df["Item"].isin(["Smoothie","Sandwich"]) 
    & 
    df["Location"].isin(["In-store","Takeaway"])
    &
    df["Payment Method"].isin(["Credit Card","Cash","Digital Wallet"])  
    )


grouped = df[known].groupby(['Item','Location','Payment Method']).size().reset_index(name='Count').sort_values(['Location','Payment Method'],ascending=[False,True])

In [ ]:
# Counting the Probability count of having sandwich or smotthie in missing item for price unit 4.0
total_count = grouped.groupby(['Location','Payment Method'])['Count'].transform("sum")
grouped['Probability'] = grouped['Count']/total_count

grouped

,Item,Location,Payment Method,Count,Probability
3,Sandwich,Takeaway,Cash,83,0.614815
9,Smoothie,Takeaway,Cash,52,0.385185
4,Sandwich,Takeaway,Credit Card,71,0.486301
10,Smoothie,Takeaway,Credit Card,75,0.513699
5,Sandwich,Takeaway,Digital Wallet,86,0.530864
11,Smoothie,Takeaway,Digital Wallet,76,0.469136
0,Sandwich,In-store,Cash,82,0.535948
6,Smoothie,In-store,Cash,71,0.464052
1,Sandwich,In-store,Credit Card,77,0.546099
7,Smoothie,In-store,Credit Card,64,0.453901


In [ ]:
# Building A Look Up Table based on probability

most_likely = (
    grouped
    .sort_values(['Location','Payment Method','Probability'],ascending=[True,True,False])
    .drop_duplicates(subset=['Location','Payment Method'])
    .set_index(['Location','Payment Method'])['Item']
)

most_likely

Location  Payment Method
In-store  Cash              Sandwich
          Credit Card       Sandwich
          Digital Wallet    Sandwich
Takeaway  Cash              Sandwich
          Credit Card       Smoothie
          Digital Wallet    Sandwich
Name: Item, dtype: object

In [88]:
# Resetting the Index and Rename Columns - (mainly to convert the series into a data_frame for merging)

lookup_df = most_likely.reset_index()
lookup_df.columns = ['Location','Payment Method','Guessed Item']

lookup_df

,Location,Payment Method,Guessed Item
0,In-store,Cash,Sandwich
1,In-store,Credit Card,Sandwich
2,In-store,Digital Wallet,Sandwich
3,Takeaway,Cash,Sandwich
4,Takeaway,Credit Card,Smoothie
5,Takeaway,Digital Wallet,Sandwich


In [89]:
# Extracting the rows where we want to fill

mask = df['Item'].isna() & (df['Price Per Unit']==4.0)
to_fill = df[mask].copy()

In [90]:
# Merging with the Lookup Table

filled = to_fill.merge(lookup_df, on=['Payment Method','Location'], how='left')
df.loc[mask,'Item'] = filled['Guessed Item'].values

In [91]:
print(f"The Total NaN values in Item: {df['Item'].isna().sum()}")

The Total NaN values in Item: 386


#### Filling the NaN values for Price per unit - 3.0

In [92]:
# Filtering Out - Juice and Cake 
known1 = (
    df['Item'].isin(['Juice','Cake'])
    &
    df['Location'].isin(['In-store','Takeaway'])
    &
    df['Payment Method'].isin(['Cash','Credit Card','Digital Wallet'])
    )

grouped1 = df[known1].groupby(['Item','Location','Payment Method']).size().reset_index(name='Count').sort_values(['Location','Payment Method'], ascending=[False,True])

grouped1

,Item,Location,Payment Method,Count
3,Cake,Takeaway,Cash,83
9,Juice,Takeaway,Cash,79
4,Cake,Takeaway,Credit Card,86
10,Juice,Takeaway,Credit Card,74
5,Cake,Takeaway,Digital Wallet,70
11,Juice,Takeaway,Digital Wallet,90
0,Cake,In-store,Cash,83
6,Juice,In-store,Cash,87
1,Cake,In-store,Credit Card,61
7,Juice,In-store,Credit Card,77


In [93]:
# Counting the Probability
total_count = grouped1.groupby(['Location','Payment Method'])['Count'].transform('sum')
grouped1['Probability'] = grouped1['Count']/total_count

grouped1

,Item,Location,Payment Method,Count,Probability
3,Cake,Takeaway,Cash,83,0.512346
9,Juice,Takeaway,Cash,79,0.487654
4,Cake,Takeaway,Credit Card,86,0.537500
10,Juice,Takeaway,Credit Card,74,0.462500
5,Cake,Takeaway,Digital Wallet,70,0.437500
11,Juice,Takeaway,Digital Wallet,90,0.562500
0,Cake,In-store,Cash,83,0.488235
6,Juice,In-store,Cash,87,0.511765
1,Cake,In-store,Credit Card,61,0.442029
7,Juice,In-store,Credit Card,77,0.557971


In [94]:
# Lookup Table
most_likely = (
    grouped1
    .sort_values(['Location','Payment Method','Probability'],ascending=[True,True,False])
    .drop_duplicates(subset=['Location','Payment Method'])
    .set_index(["Location",'Payment Method'])['Item']
)
most_likely

Location  Payment Method
In-store  Cash              Juice
          Credit Card       Juice
          Digital Wallet    Juice
Takeaway  Cash               Cake
          Credit Card        Cake
          Digital Wallet    Juice
Name: Item, dtype: object

In [95]:
# Resetting The Index
lookup_df = most_likely.reset_index()
lookup_df.columns = ['Location','Payment Method','Guessed Item']

lookup_df

,Location,Payment Method,Guessed Item
0,In-store,Cash,Juice
1,In-store,Credit Card,Juice
2,In-store,Digital Wallet,Juice
3,Takeaway,Cash,Cake
4,Takeaway,Credit Card,Cake
5,Takeaway,Digital Wallet,Juice


In [96]:
# Extract only the rows where we want to fill
mask1 = df['Item'].isna() & (df['Price Per Unit']==3.0)
to_fill1 = df[mask1].copy()


In [97]:
# Merging with the Lookup Table
filled = to_fill1.merge(lookup_df, on=['Payment Method','Location'], how='left')
df.loc[mask1,'Item'] = filled['Guessed Item'].values

In [98]:
print(f"The Total NaN values in Item: {df['Item'].isna().sum()}")

The Total NaN values in Item: 279


##### Handling the NaN values in Payment Method

In [99]:
print(f"Total NaN values in Payment Method: {df['Payment Method'].isna().sum()}")

Total NaN values in Payment Method: 3178


In [100]:
known_pay = (
    df['Payment Method'].isin(["Cash","Credit Card","Digital Wallet"])
    &
    df["Location"].isin(['In-store','Takeaway'])
    &
    df["Item"].isin(['Coffee','Salad','Cake','Cookie','Smoothie','Sandwich','Juice','Tea'])          
    )

grouped_pay = (
    df[known_pay].groupby(['Item','Location','Payment Method']).aggregate(Count=('Item','size'),Total_Quantity=('Quantity','sum'))
    .reset_index()
    .sort_values(['Item','Payment Method'],ascending=[False,False])
)

In [ ]:
## Calculating two probabilties, one is total count and another one is based on quantity
total_count_pay = grouped_pay.groupby(['Item','Location'])['Count'].transform('sum')
grouped_pay['Transaction Probability'] = grouped_pay["Count"]/total_count_pay

total_quantity_pay = grouped_pay.groupby(['Item','Location'])['Total_Quantity'].transform('sum')
grouped_pay["Quantity Probability"] = grouped_pay['Total_Quantity']/ total_quantity_pay

In [ ]:
## Combining two Probabilities 
grouped_pay['Probability'] = (grouped_pay['Transaction Probability'] + grouped_pay['Quantity Probability'])/2

In [103]:
## Lookup Table
most_likely_pay = (
    grouped_pay.sort_values(['Item','Location','Probability'],ascending=[True,True,False])
    .drop_duplicates(subset=['Item','Location'])
    .set_index(["Item",'Location'])['Payment Method']
)

most_likely_pay

Item      Location
Cake      In-store              Cash
          Takeaway       Credit Card
Coffee    In-store       Credit Card
          Takeaway    Digital Wallet
Cookie    In-store       Credit Card
          Takeaway    Digital Wallet
Juice     In-store              Cash
          Takeaway    Digital Wallet
Salad     In-store              Cash
          Takeaway       Credit Card
Sandwich  In-store    Digital Wallet
          Takeaway    Digital Wallet
Smoothie  In-store    Digital Wallet
          Takeaway       Credit Card
Tea       In-store    Digital Wallet
          Takeaway       Credit Card
Name: Payment Method, dtype: object

In [104]:
lookup_df_pay = most_likely_pay.reset_index()
lookup_df_pay.columns = ['Item','Location','Guessed Payment Method']

In [105]:
mask_pay = df['Payment Method'].isna()
to_fill_pay = df[mask_pay].copy()

In [106]:
filled_pay = to_fill_pay.merge(lookup_df_pay, on=['Item','Location'], how='left')

df.loc[mask_pay,'Payment Method'] = filled_pay['Guessed Payment Method'].values

In [107]:
print(f"Total NaN values in Payment Method: {df['Payment Method'].isna().sum()}")

Total NaN values in Payment Method: 1380


#### Handling the NaN value in Location

In [108]:
print(f'The Total NaN values in Location: {df['Location'].isna().sum()}')

The Total NaN values in Location: 3961


In [109]:
known_location = (
    df['Location'].isin(['In-store','Takeaway'])
    &
    df['Payment Method'].isin(['Cash','Credit Card','Digital Wallet'])
    &
    df['Item'].isin(['Sandwich','Smoothie','Cake','Cookie','Salad','Coffee','Juice','Tea'])
    )

grouped_location = (
    df[known_location].groupby(['Item','Payment Method','Location'])
    .aggregate(
        Count = ('Item','size'),
        total_quantity_location = ('Quantity','sum')
    )
    .reset_index()
    .sort_values(['Item','Payment Method'],ascending=[False,False])
)

In [110]:
total_count_location = grouped_location.groupby(['Item','Payment Method'])['Count'].transform('sum')
grouped_location['Location Probability'] = grouped_location['Count']/total_count_location

total_quantity_location = grouped_location.groupby(['Item','Payment Method'])['total_quantity_location'].transform('sum')
grouped_location['Quantity Probability'] = grouped_location['total_quantity_location']/total_quantity_location

In [111]:
grouped_location['Combined Location']= (grouped_location['Location Probability'] + grouped_location['Quantity Probability'])/2

In [112]:
most_likely_location = (
    grouped_location.sort_values(['Item','Payment Method','Combined Location'],ascending=[False,False,False])
    .drop_duplicates(subset=['Item','Payment Method'])
    .set_index(['Item','Payment Method'])['Location']
)

In [113]:
lookup_df_location = most_likely_location.reset_index()
lookup_df_location.columns = ['Item','Payment Method','Guessed Location']

In [114]:
mask_location = df['Location'].isna()
to_fill_location = df[mask_location].copy()

In [115]:
filled_location = to_fill_location.merge(lookup_df_location,on=['Item','Payment Method'], how='left')
df.loc[mask_location,'Location'] = filled_location['Guessed Location'].values

In [116]:
print(f"The total NaN value in Location: {df['Location'].isna().sum()}")

The total NaN value in Location: 1425


#### Removing the Duplicates

In [117]:
print(f'Total number of duplicates: {df.duplicated().sum()}')

Total number of duplicates: 743


In [118]:
df = df.drop_duplicates()
print(f'Total number of duplicates: {df.duplicated().sum()}')

Total number of duplicates: 0


#### Dropping the remaining NaN values

In [119]:
df = df.dropna(subset=['Item','Price Per Unit','Quantity','Payment Method','Location'])

#### Saving the cleaned file

In [120]:
df.to_csv('../data/processed/cleaned_cafe_sales.csv', index=False)
print("Cleaned data exported successfully.")

Cleaned data exported successfully.
